In [1]:
import yaml
import logging

import pandas as pd 

from LM import LLM,JudgeLLM
from utils import load_environment,dump_in_jsonl

load_environment()

In [2]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

logger = logging.getLogger(__name__)

In [3]:
with open("models.yaml","r") as f:
    models = yaml.safe_load(f)

test_models = models['TEST_MODELS']
judge_models = models['JUDGE_MODELS']

In [4]:
LIMIT_SAMPLE_SIZE = 1

df = pd.read_csv("../data/examples.csv")
df = df[:LIMIT_SAMPLE_SIZE]

evaluation_prompts_file_path = "../data/evaluation_prompts.csv"

In [5]:
model_instance = LLM(test_models[0])
test_model_output = model_instance.call_llm(
    df['full_prompt'].iloc[0],intention="test",sample_id = 0
)

test_model_output

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:LM:Successfully called : openai:gpt-3.5-turbo | Intention : test.


{'sample_id': 0,
 'test model': 'openai:gpt-3.5-turbo',
 'Encountered Problems': False,
 'test model response': 'Based on the information provided in the context document, your risk factors for dementia include high blood pressure, diabetes, obesity, lack of physical activity, poor diet, high alcohol consumption, low cognitive engagement, depression, traumatic brain injury, hearing loss, and social isolation. \n\nCognitive engagement is important because it is thought to support the development of a "cognitive reserve," which may protect against brain cell damage caused by dementia. Engaging in activities that challenge and stimulate the brain throughout life may help reduce the risk of developing dementia.'}

In [6]:
dump_in_jsonl(test_model_output,"test_responses.jsonl")


# Judge 

In [7]:
judges = JudgeLLM(
     judge_models
)

judges

In [8]:
output = judges.calling_judges(
    user_request= df['user_request'].iloc[0],
    context_document= df['context_document'].iloc[0],
    test_model_response= test_model_output['test model response'],
    test_model= test_model_output['test model'],
    sample_id= 0, 
    evaluation_prompt_file_path=evaluation_prompts_file_path,
)

INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:LM:Successfully called : anthropic:claude-3-5-sonnet-20240620 | Intention : judge.
INFO:LM:Accurate
INFO:LM:Successfully called : google_vertexai:gemini-1.5-pro | Intention : judge.
INFO:LM:Accurate


In [9]:
output

[{'judged_model': 'openai:gpt-3.5-turbo',
  'sample_id': 0,
  'judge model': 'anthropic:claude-3-5-sonnet-20240620',
  'Encountered Problems': False,
  'judge model response': 'Sentence 1: Based on the information provided in the context document, your risk factors for dementia include high blood pressure, diabetes, obesity, lack of physical activity, poor diet, high alcohol consumption, low cognitive engagement, depression, traumatic brain injury, hearing loss, and social isolation.\nSentence 1 label: Accurate\n\nSentence 2: Cognitive engagement is important because it is thought to support the development of a "cognitive reserve," which may protect against brain cell damage caused by dementia.\nSentence 2 label: Accurate\n\nSentence 3: Engaging in activities that challenge and stimulate the brain throughout life may help reduce the risk of developing dementia.\nSentence 3 label: Accurate\n\nFinal Answer: Accurate',
  'verdict': 'Accurate'},
 {'judged_model': 'openai:gpt-3.5-turbo',
 

# to_dict 

In [1]:
import pandas as pd 

In [3]:
df = pd.read_csv("../data/examples.csv")[:2]
df

,system_instruction,user_request,context_document,full_prompt,domain,type,high_level_type
0,"{instruction}\n ==========\n In your answer, r...","I'm middle-aged, never smoked, had my ears blo...",high blood pressure \n People who have consist...,"{instruction}\n ==========\n In your answer, r...",Medical,Effect Analysis,Q&A
1,You formulate answers based solely on the mate...,Can you list all the knife brands that sell kn...,Knife sharpening angles\nManufacturer´s recomm...,You formulate answers based solely on the mate...,Retail/Product,Fact Finding,Q&A


In [4]:
records = df.to_dict(orient='records')
records

[{'system_instruction': '{instruction}\n ==========\n In your answer, refer only to the context document. Do not employ any outside knowledge\n \n\n {question}\n ==========\n [user request]\n \n\n {passage 0}\n ==========\n [context document]',
  'user_request': "I'm middle-aged, never smoked, had my ears blown out in the war, get a case of the sads pretty regular, and eat mostly garbage. What are my risk factors for dementia? What does cognitive engagement have to do with it?",
  'context_document': 'high blood pressure \n People who have consistent high blood pressure (hypertension) in mid-life (ages 45 to 65) are more likely to develop dementia compared to those with normal blood pressure.\n \n\n High blood pressure can increase the risk of developing dementia, particularly vascular dementia, because of its effect on the heart, the arteries, and blood circulation.\n \n\n Smoking \n The evidence is strong and consistent that smokers are at a higher risk of developing dementia vs. non

In [9]:
for sample_id,record in enumerate(records):
    print(sample_id)
    print(record['type'])
    print("--"*30)

0
Effect Analysis
------------------------------------------------------------
1
Fact Finding
------------------------------------------------------------
